In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, WebDriverException
from bs4 import BeautifulSoup as BS4
from scrapy import spiders
from scrapy.spiders import CrawlSpider, Rule

import scrapy
import yaml
import getpass
import time
import json
import pandas as pd


In [2]:
# Locations

df = pd.read_csv('./uk-towns-sample.csv')
location = list(dict.fromkeys(df['county'] + ', ' + df['country']))
location.remove('County Derry / Londonderry, Northern Ireland')
location

['Surrey, England',
 'Somerset, England',
 'Worcestershire, England',
 'Essex, England',
 'Northumberland, England',
 'Devon, England',
 'Powys, Wales',
 'Gloucestershire, England',
 'South Yorkshire, England',
 'Herefordshire, England',
 'Warwickshire, England',
 'Shropshire, England',
 'Staffordshire, England',
 'Greater Manchester, England',
 'City of Edinburgh, Scotland',
 'Wiltshire, England',
 'Fife, Scotland',
 'Berwickshire, Scotland',
 'Lancashire, England',
 'Cumbria, England',
 'Greater London, England',
 'The Stewartry of Kirkcudbright, Scotland',
 'Roxburgh, Ettrick and Lauderdale, Scotland',
 'Dorset, England',
 'West Sussex, England',
 'Hertfordshire, England',
 'Cambridgeshire, England',
 'Cheshire, England',
 'Hampshire, England',
 'Dyfed, Wales',
 'Mid Glamorgan, Wales',
 'Gwynedd, Wales',
 'Inverness, Scotland',
 'Perth and Kinross, Scotland',
 'West Glamorgan, Wales',
 'Gwent, Wales',
 'Banffshire, Scotland',
 'West Lothian, Scotland',
 'City of Aberdeen, Scotland',

In [3]:
# Login to Linked In

driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
actions = ActionChains(driver)

driver.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

email = driver.find_element_by_id('username')
email.clear()
email.send_keys('krunaldholakia01@gmail.com')

password = driver.find_element_by_id('password')
password.clear()
password.send_keys('A@1@bdc@543')

driver.find_element_by_xpath('//div[@class="login__form_action_container "]/button').click()

time.sleep(3)

driver.find_element_by_xpath('//a[@href="/jobs/"]').click()

time.sleep(3)

In [4]:
# Keywords Input
keywords = 'data scientist'
jobKeywords = driver.find_element_by_xpath('//*[@aria-label="Search by title, skill, or company"]')
jobKeywords.clear()
jobKeywords.send_keys(keywords)


In [5]:
# Location Input

jobList = []

for loc in location:
    # Search by Locations
    jobLocation = driver.find_element_by_xpath('//*[@aria-label="City, state, or zip code"]')
    jobLocation.clear()
    jobLocation.send_keys(loc)
    time.sleep(3)
    jobLocation.send_keys(Keys.RETURN)
    print('Scraping %s' %loc)
    time.sleep(5)
    for i in range(1, 25):
        try:
            print('Page %d' %i)

            for job in driver.find_elements_by_xpath('//ul[@itemtype="http://schema.org/ItemList"]/li'):
                # Fetching Job Post Link and Locations from List
                driver.execute_script("arguments[0].scrollIntoView();", job)
                jobInfo = {}
                jobInfo['link'] = job.find_element_by_xpath('./div/div/div[1]/div[2]/div[1]/a').get_attribute('href')
                jobInfo['location'] = job.find_element_by_xpath('./div/div/div[1]/div[2]/div[3]/ul/li').text
                
                jobList.append(jobInfo)
                                       
            try:
                # Job List Pages handling
                count = len(driver.find_elements_by_xpath('//*[@class="jobs-search-two-pane__pagination"]/div/ul/li'))
                if count > i:
                    driver.find_element_by_xpath('//*[@class="jobs-search-two-pane__pagination"]/div/ul/li['+str(i + 1)+']/button').click()
                
                else:
                    break
                
            except Exception as e:
                print(e)
            
            print('Page %d Finished' %i)
            
        except Exception as e:
            print(e)
            
        time.sleep(3)
        
    # Writing Location jobs into JSON file
    with open('./J/%s.json' %loc.replace(',', ''), 'w') as fp:
        json.dump(jobList, fp)
        jobList = []
        
info = []
# links = 

Scraping Surrey, England
Page 1
Page 1 Finished
Page 2
Page 2 Finished
Page 3
Page 3 Finished
Page 4
Page 4 Finished
Page 5
Page 5 Finished
Page 6
Page 6 Finished
Page 7
Scraping Somerset, England
Page 1
Message: element click intercepted: Element <button aria-label="Page 2" type="button" data-ember-action="" data-ember-action-3049="3049">...</button> is not clickable at point (560, 690). Other element would receive the click: <div aria-label="search results" aria-describedby="results-list__title" class="jobs-search-two-pane__results">...</div>
  (Session info: chrome=92.0.4515.159)

Page 1 Finished
Page 2
Scraping Worcestershire, England
Page 1
Page 1 Finished
Page 2
Scraping Essex, England
Page 1
Page 1 Finished
Page 2
Page 2 Finished
Page 3
Scraping Northumberland, England
Page 1
Scraping Devon, England
Page 1
Page 1 Finished
Page 2
Page 2 Finished
Page 3
Scraping Powys, Wales
Page 1
Scraping Gloucestershire, England
Page 1
Page 1 Finished
Page 2
Page 2 Finished
Page 3
Page 3 Finish

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=92.0.4515.159)
